Calculation of price elasticity in the semoPX Day-ahead market using order book data.

In [1]:
import os
import pandas as pd
from lxml import etree

data = []
folder_path = r"C:\Users\honan\OneDrive\Desktop\SEMO\BidAskCurves"
for file_name in os.listdir(folder_path):
    if file_name.endswith(".xml"):
        file_path = os.path.join(folder_path, file_name)
        try:
            with open(file_path, "r") as f:
                xml_str = f.read()
            root = etree.fromstring(xml_str.encode())
            for MarketArea in root.iter("MarketArea"):
                MarketAreaName = MarketArea.find("MarketAreaName").text
                DeliveryDay = MarketArea.find("DeliveryDay/Day").text
                for t_step in MarketArea.iter("TimeStep"):
                    TimeStep = t_step.find("TimeStepID").text
                    for purchase in t_step.iter("Purchase"):
                        Price = purchase.find("Price").text
                        Volume = purchase.find("Volume").text
                        data.append({
                            "MarketArea":MarketAreaName,
                            "DeliveryDay":DeliveryDay,
                            "TimeStep":TimeStep,
                            "Price":Price,
                            "Volume":Volume,
                            "Type":"Purchase"
                        })
                    for sell in t_step.iter("Sell"):
                        Price = sell.find("Price").text
                        Volume = sell.find("Volume").text
                        data.append({
                            "MarketArea":MarketAreaName,
                            "DeliveryDay":DeliveryDay,
                            "TimeStep":TimeStep,
                            "Price":Price,
                            "Volume":Volume,
                            "Type":"Sell"
                        })
            print(f'Successfully read {file_name}')
        except:
            print(f'Error reading {file_name}, skipping...')
            continue

df = pd.DataFrame(data)
df.head()


Successfully read BidAskCurves_NI-DA_20181001_20180930131435.xml
Successfully read BidAskCurves_NI-DA_20181002_20181001131656.xml
Successfully read BidAskCurves_NI-DA_20181003_20181002131637.xml
Successfully read BidAskCurves_NI-DA_20181004_20181003131153.xml
Successfully read BidAskCurves_NI-DA_20181005_20181004131802.xml
Successfully read BidAskCurves_NI-DA_20181006_20181005131447.xml
Successfully read BidAskCurves_NI-DA_20181007_20181006131258.xml
Successfully read BidAskCurves_NI-DA_20181008_20181007131355.xml
Successfully read BidAskCurves_NI-DA_20181009_20181008131715.xml
Successfully read BidAskCurves_NI-DA_20181010_20181009131237.xml
Successfully read BidAskCurves_NI-DA_20181011_20181010131502.xml
Successfully read BidAskCurves_NI-DA_20181012_20181011131639.xml
Successfully read BidAskCurves_NI-DA_20181013_20181012131523.xml
Successfully read BidAskCurves_NI-DA_20181014_20181013131407.xml
Successfully read BidAskCurves_NI-DA_20181015_20181014131432.xml
Successfully read BidAskC

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43896802 entries, 0 to 43896801
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   MarketArea   object
 1   DeliveryDay  object
 2   TimeStep     object
 3   Price        object
 4   Volume       object
 5   Type         object
dtypes: object(6)
memory usage: 2.0+ GB


In [ ]:
df.tail()

,MarketArea,DeliveryDay,TimeStep,Price,Volume,Type
43896797,SEM-IDA3,29/05/2022,23HH2,725,1055.2,Sell
43896798,SEM-IDA3,29/05/2022,23HH2,725,1105.2,Sell
43896799,SEM-IDA3,29/05/2022,23HH2,2000,1105.2,Sell
43896800,SEM-IDA3,29/05/2022,23HH2,2000,1112.2,Sell
43896801,SEM-IDA3,29/05/2022,23HH2,3000,1112.2,Sell


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression


# Create an empty list to store the results
results = []

# Loop through each unique combination of MarketArea, delivery day, and timestep
for market_area in df['MarketArea'].unique():
    for delivery_day in df['DeliveryDay'].unique():
        for time_step in df['TimeStep'].unique():
            # Filter the data to include only the current combination of MarketArea, delivery day, and timestep
            subset = df[(df['MarketArea'] == market_area) & (df['DeliveryDay'] == delivery_day) & (df['TimeStep'] == time_step)]

            # If the subset is not empty, calculate the price elasticity
            if not subset.empty:
                # Calculate the average price and volume for the subset
                avg_price = subset['Price'].mean()
                avg_volume = subset['Volume'].mean()

                # Use linear regression to estimate the demand/supply function for the subset
                X = subset['Price'].values.reshape(-1, 1)
                y = subset['Volume'].values.reshape(-1, 1)
                regressor = LinearRegression()
                regressor.fit(X, y)
                elasticity = -regressor.coef_[0] * (avg_price / avg_volume)

                # Append the result to the results list
                result = {'MarketArea': market_area,
                          'DeliveryDay': delivery_day,
                          'TimeStep': time_step,
                          'PriceElasticity': elasticity}
                results.append(result)

# Create a pandas dataframe from the list of dictionaries
df_results = pd.DataFrame(results)

# Print the dataframe
print(df_results)


   MarketArea DeliveryDay TimeStep          PriceElasticity
0    SEM-IDA3  19/01/2023    18HH1  [-0.039746403091553605]
1    SEM-IDA3  19/01/2023    18HH2   [-0.06679900183049012]
2    SEM-IDA3  19/01/2023    19HH1  [-0.039135724067941274]
3    SEM-IDA3  19/01/2023    19HH2     [-0.083720742707274]
4    SEM-IDA3  19/01/2023    20HH1   [-0.04994686571117935]
5    SEM-IDA3  19/01/2023    20HH2   [-0.08404363318669218]
6    SEM-IDA3  19/01/2023    21HH1   [-0.08385337731371473]
7    SEM-IDA3  19/01/2023    21HH2   [-0.11325563671525397]
8    SEM-IDA3  19/01/2023    22HH1   [-0.16822591481852078]
9    SEM-IDA3  19/01/2023    22HH2   [-0.15752216629428337]
10   SEM-IDA3  19/01/2023    23HH1   [-0.16156911602966806]
11   SEM-IDA3  19/01/2023    23HH2   [-0.17015148732788918]


In [ ]:
df_results.info()